In [3]:
import numpy as np
from gpkit import Model, Variable, units
from gpkit.shortcuts import Var
from gpkit.tools import te_exp_minus1

In [119]:
class BreguetRange(Model):
    def setup(self):
        g = Var("g", 9.81, "m/s^2", "gravity")
        R = Var("R", "nautical_miles", "range")
        T = Var("T", "N", "Thrust")
        h_fuel = Var("h_{fuel}", 42e6, "J/kg", "fuel heating value")
        eta_0 = Var("\\eta_0", 0.4, "-", "Overall engine efficiency")
        W = Var("W", "N", "Aircraft weight")
        W_zfw = Var("W_{zfw}", "N", "Zero fuel Weight")
        W_fuel = Var("W_{fuel}", "N", "Fuel weight")
        z_bre = Var("z_{bre}", "-", "Breguet parameter")

        # Set up Model Equations
        objective = 1/R  # Maximize range
        constraints = [W >= W_zfw + W_fuel,
                       z_bre >= g*R*T/(h_fuel*eta_0*W),
                       W_fuel/W >= te_exp_minus1(z_bre, nterm=3)
                      ]
        return objective, constraints
    
    def test(self):
        self.substitutions.update({"W_{zfw}":400, "W":500, "T":15})
        self.solve(verbosity=0)

BreguetRange().test()

In [120]:
class Cruise(Model):
    def setup(self):
        W = Var("W", "N", "Aircraft weight")        
        T = Var("T", "N", "Thrust")
        V = Var("V", "m/s", "Velocity")
        C_L = Var("C_L", "-", "Lift coefficient, aircraft")
        C_D = Var("C_D", "-", "Drag coefficient, aircraft")
        S_w = Var("S_w", "m^2", "Area, wing")
        rho_fa = Var("rho_{fa}", 0.9, "kg/m^3", "Air density, at flight altitutde")
        
        constraints = [T >= 0.5*C_D*V**2*S_w*rho_fa,
                       W == 0.5*C_L*V**2*S_w*rho_fa]
        objective = T/W
        
        return objective, constraints
        
    def test(self):
        self.substitutions.update({"V":35, "T":15, "S_w":5, "C_D":0.005, "C_L":1})
        self.solve(verbosity=0)
        
Cruise().test()

In [159]:
class VTOL(Model):
    def setup(self):
        m_esys = Var("m_{esys}", "g", "Electrical Systems Weight")
        m = Var("m", "g", "Mass at full fuel")
        m_min = Var("m_{min}", 50, "g", "Minimum mass")
        m_max = Var("m_{max}", 10000, "g", "Maximum mass")
        g = Var("g", 9.81, "m/s^2", "gravity")

        
        constraints = [m_esys == 308.3*m**0.5087*units('g/g^0.5087'),
                       m >= m_min,
                       m <= m_max]
        objective = m_esys
        
        return objective, constraints
    
    def test(self):
        self.substitutions.update({"m":1000})
        self.solve(verbosity=0)
        
VTOL().test()

In [164]:
class Sizing(Model):
    def setup(self):
        Range = BreguetRange()
        cruise = Cruise()
        vtol = VTOL()
        
        W = Range["W"]
        W_zfw = Range["W_{zfw}"]
        R = Range["R"]
        
        C_D = cruise["C_D"]
        C_L = cruise["C_L"]
        S_w = cruise["S_w"]
        V = cruise["V"]
        
        m_esys = vtol["m_{esys}"]
        m = vtol["m"]
        
        g = Var("g", 9.81, "m/s^2", "gravity")
        m_pay = Var("m_{pay}", 5, "kg", "Payload Mass")
        #W_mto = Var("W_{mto}", 500, "N", "Mass take off weight")
        W_mto = Var("W_{mto}", 500, "N", "Mass take off weight")
        W_zfwmax = Var("W_{zfwmax}", 200, "N", "Max zfw")
        C_Dmin = Var("C_{Dmin}", 0.005, "-", "CD min")
        S_wmin = Var("S_{swmin}", 5, "m^2", "Surface area min")
        V_min = Var("V_{min}", 35, "m/s", "Min velocity")
        
        
        constraints = [C_D >= C_Dmin,
                       S_w >= S_wmin,
                       V >= V_min,
                       W_zfw >= W_zfwmax,
                       m_esys*g >= W_zfw,
                       W >= m*g + m_esys*g,
                       W_mto >= m_esys*g + W + m_pay*g]
        cost = 1/R
        model = Model(cost, constraints)
        
        for subm in [Range, cruise, vtol]:
            model = model & subm
        
        return model
    
    def test(self):
        self.solve(verbosity=0)
        
print Sizing().solve(verbosity=0).table(["cost", "freevariables"])


Cost
----
 0.0003212 [1/nmi] 

Free Variables
--------------
               Sizing84 |                                             
                    C_D : 0.005             Drag coefficient, aircraft
                      R : 3113       [nmi]  range                     
                    S_w : 5          [m**2] Area, wing                
                      T : 13.78      [N]    Thrust                    
                      V : 35         [m/s]  Velocity                  
                      W : 251        [N]    Aircraft weight           
                W_{zfw} : 200        [N]    Zero fuel Weight          
                      m : 3789       [g]    Mass at full fuel         
               m_{esys} : 2.039e+04  [g]    Electrical Systems Weight 
                                                                      
Sizing84BreguetRange100 |                                             
                      R : 3113       [nmi]  range                     
               